# Assistant Feature Specification

## 1. Purpose

The Assistant feature provides a voice-based AI assistant that enables users to:
- Have natural conversations with an LLM through voice and text
- Interrupt the assistant mid-response when needed
- See a synchronized chat transcript of the conversation
- Resume conversation from the exact point of interruption

This creates a more natural conversation experience, similar to talking with a human, where users can interrupt and redirect the conversation.

## 2. Key Functionality

### Core Components

1. **AssistantBloc**: Central coordinator managing state and events
   - Handles OpenAI client connection and events
   - Manages voice recording and playback states
   - Coordinates with ChatBloc for message display

2. **AudioService**: Manages audio playback with position tracking
   - Streams audio chunks with item identification
   - Provides interruption capability with position information
   - Supports resuming from interruption point

3. **PositionTracker**: Precise audio position tracking
   - Tracks audio chunks and sample counts
   - Accounts for pauses and buffering
   - Provides accurate interruption positions

### Key States

```dart
enum ClientStatus { connecting, ready, rateLimited, error }
enum UserInputState { idle, recording, recorded }
enum ResponseState { idle, responding }
enum VoiceStreamState { idle, playing, error }
```

### Flow Overview

1. **User Input Flow**:
   - Record audio (up to 30 seconds)
   - Send to OpenAI for transcription
   - Display transcribed message in chat

2. **Assistant Response Flow**:
   - Receive text and audio chunks from OpenAI
   - Display text in chat incrementally
   - Stream audio for playback
   - Track playback position for potential interruption

3. **Interruption Flow**:
   - User taps interrupt button
   - Get current position (itemId + sampleCount) from PositionTracker
   - Stops voice playback
   - Send interruption to OpenAI with position
   - Update UI to show interruption point
   - Allow conversation to continue from that point


## 3. Usage Examples

### Setting up the AssistantBloc

```dart
class AssistantBloc extends Bloc<AssistantEvent, AssistantState> {
  final ChatBloc _chatBloc;
  final AudioService _audioService;
  final AudioRecorder _recorder;
  final RealtimeClient _client;
  final PositionTracker _positionTracker;
  
  AssistantBloc({
    required ChatBloc chatBloc,
    required AudioService audioService,
    required AudioRecorder recorder,
  }) : _chatBloc = chatBloc,
       _audioService = audioService,
       _recorder = recorder,
       _positionTracker = StreamTimeline(24000), // 24kHz for voice
       _client = RealtimeClient(
         apiKey: EnvConfig.openAiKey,
         dangerouslyAllowAPIKeyInBrowser: true,
       ) {
    _setupClient();
    _registerEventHandlers();
  }
  
  Future<void> _setupClient() async {
    // Initialize OpenAI client
    await _client.updateSession(
      instructions: 'You are a friendly meditation assistant.',
      voice: Voice.alloy,
    );
    
    // Register event handlers
    _client.on(RealtimeEventType.conversationUpdated, _handleConversationUpdate);
    _client.on(RealtimeEventType.conversationInterrupted, _handleInterruption);
    
    // Connect to OpenAI
    await _client.connect();
    add(ClientConnected());
  }
  
  void _registerEventHandlers() {
    on<StartRecordingUserAudioInput>(_onStartRecording);
    on<StopRecordingUserAudioInput>(_onStopRecording);
    on<SendRecordedAudio>(_onSendRecordedAudio);
    on<InterruptResponse>(_onInterruptResponse);
    // client.on(RealtimeEventType.conversationUpdated creates evenst 
    // Additional event handlers...
  }
}
```

### Handling Voice Recording and Sending

```dart
Future<void> _onStartRecording(
  StartRecordingUserAudioInput event,
  Emitter<AssistantState> emit,
) async {
  if (!state.canRecord) return;
  
  await _recorder.startRecording();
  emit(state.copyWith(userInput: UserInputState.recording));
  
  // Start duration timer
  _recordingTimer = Timer.periodic(
    const Duration(milliseconds: 100),
    (timer) {
      final newDuration = state.recordingDuration + const Duration(milliseconds: 100);
      if (newDuration >= AssistantState.maxRecordingDuration) {
        add(StopRecordingUserAudioInput());
      } else {
        emit(state.copyWith(recordingDuration: newDuration));
      }
    },
  );
}

Future<void> _onSendRecordedAudio(
  SendRecordedAudio event,
  Emitter<AssistantState> emit,
) async {
  if (!state.canSendRecording || state.recordedAudio == null) return;
  
  final base64Audio = base64Encode(state.recordedAudio!);
  
  // Send to OpenAI
  await _client.sendUserMessageContent([
    ContentPart.inputAudio(audio: base64Audio),
  ]);
  
  // Reset recording state
  emit(state.copyWith(
    userInput: UserInputState.idle,
    recordedAudio: null,
    recordingDuration: Duration.zero,
  ));
}
```

### Processing Assistant Responses

```dart
void _handleConversationUpdate(RealtimeEvent event) {
  final update = event as RealtimeEventConversationUpdated;
  final result = update.result;
  final item = result.item;
  final delta = result.delta;
  
  if (item == null) return;
  
  // Handle user message transcription
  if (item.item case ItemMessage message when message.role == ItemRole.user) {
    if (delta?.transcript != null) {
      add(UserMessageTranscribed(
        itemId: item.id,
        transcript: delta!.transcript!,
      ));
    }
  }
  
  // Handle assistant response
  if (item.item case ItemMessage message when message.role == ItemRole.assistant) {
    // For text content
    if (delta?.transcript != null) {
      add(ResponseTextReceived(
        itemId: item.id,
        text: delta!.transcript!,
      ));
    }
    
    // For audio content
    if (delta?.audio != null) {
      final audioData = base64Decode(delta!.audio!);
      add(ResponseAudioReceived(
        itemId: item.id,
        audioData: audioData,
      ));
    }
  }
}

Future<void> _onResponseAudioReceived(
  ResponseAudioReceived event,
  Emitter<AssistantState> emit,
) async {
  // Add chunk to position tracker
  _positionTracker.addChunk(event.itemId, event.audioData.length);
  
  // Send to audio service for playback
  await _audioService.appendVoiceChunk(event.itemId, event.audioData);
  
  // Update state if not already responding
  if (state.responseState != ResponseState.responding) {
    emit(state.copyWith(responseState: ResponseState.responding));
  }
}
```

### Handling Interruptions

```dart
Future<void> _onInterruptResponse(
  InterruptResponse event,
  Emitter<AssistantState> emit,
) async {
  if (state.responseState != ResponseState.responding) return;
  
  // Get current position from tracker
  final timestamp = DateTime.now().millisecondsSinceEpoch;
  final result = _positionTracker.getInterruptionState(timestamp);
  
  // Stop audio playback
  await _audioService.stopVoice();
  
  // Tell OpenAI about interruption
  await _client.cancelResponse(result.itemId, result.sampleCount);
  
  // Update UI to show interruption
  _chatBloc.add(MarkMessageAsInterrupted(
    id: result.itemId,
    interruptedAt: result.sampleCount,
  ));
  
  // Update state
  emit(state.copyWith(
    responseState: ResponseState.interrupted,
    lastInterruption: result,
  ));
  
  // Reset trackers
  _positionTracker.reset();
}
```

### UI Integration Example

```dart
class AssistantView extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return BlocBuilder<AssistantBloc, AssistantState>(
      builder: (context, state) {
        return Column(
          children: [
            Expanded(
              child: ChatWidget(), // Shows conversation history
            ),
            _buildControls(context, state),
          ],
        );
      },
    );
  }
  
  Widget _buildControls(BuildContext context, AssistantState state) {
    // Show recording controls
    if (state.userInput == UserInputState.recording) {
      return _buildRecordingControls(context, state);
    }
    
    // Show send controls after recording
    if (state.userInput == UserInputState.recorded) {
      return _buildSendControls(context, state);
    }
    
    // Show interrupt button during response
    if (state.responseState == ResponseState.responding) {
      return Center(
        child: ElevatedButton.icon(
          icon: const Icon(Icons.pan_tool),
          label: const Text('Interrupt'),
          onPressed: () => context.read<AssistantBloc>()
              .add(const InterruptResponse()),
          style: ElevatedButton.styleFrom(
            backgroundColor: Colors.redAccent,
          ),
        ),
      );
    }
    
    // Default microphone button
    return Center(
      child: IconButton(
        icon: const Icon(Icons.mic),
        onPressed: state.canRecord
            ? () => context.read<AssistantBloc>()
                .add(const StartRecordingUserAudioInput())
            : null,
        iconSize: 48,
      ),
    );
  }
}
```

## 4. Test Case Descriptions

### AssistantBloc Tests

1. **Client Connection Tests**
   - `should_initialize_client_successfully`: Verify client connects to OpenAI and emits proper state
   - `should_handle_connection_failures`: Test error handling for connection issues
   - `should_handle_rate_limiting`: Verify rate limit detection and appropriate UI feedback

2. **Voice Recording Tests**
   - `should_record_and_update_duration`: Test recording state and duration updates
   - `should_stop_at_max_duration`: Verify recording stops at 30s limit
   - `should_cancel_recording`: Test recording cancellation flow
   - `should_send_recorded_audio`: Verify audio is properly encoded and sent

3. **Response Handling Tests**
   - `should_process_text_responses`: Test text chunks are passed to ChatBloc
   - `should_process_audio_responses`: Verify audio is passed to AudioService
   - `should_track_current_response_item`: Test response item ID tracking
   - `should_handle_multiple_response_items`: Test handling of sequential responses

4. **Interruption Tests**
   - `should_interrupt_response_correctly`: Verify interruption process and position capture
   - `should_send_proper_interruption_to_openai`: Test cancelResponse with correct parameters
   - `should_mark_chat_message_as_interrupted`: Check chat message updating
   - `should_update_ui_after_interruption`: Test UI state changes after interruption

### AudioService Tests

1. **Audio Streaming Tests**
   - `should_handle_initial_chunk`: Test first chunk initialization
   - `should_append_additional_chunks`: Verify multiple chunks are handled
   - `should_handle_stream_completion`: Test proper completion state

2. **Position Tracking Tests**
   - `should_track_sample_count_accurately`: Verify sample counting logic
   - `should_account_for_pauses`: Test pause handling in tracking
   - `should_track_multiple_chunks`: Verify position across chunk boundaries

3. **Stop Voice Tests**
   - `should_stop_voice_correctly`: Verify playback stops
   - `should_maintain_position_when_paused`: Test position stability during pauses
   - `should_reset_after_completion`: Verify proper cleanup

### UI Integration Tests

1. **View Tests**
   - `should_show_correct_controls_based_on_state`: Verify UI updates based on state
   - `should_show_recording_progress`: Test recording progress indicator
   - `should_disable_buttons_when_unavailable`: Check proper button state management

2. **User Interaction Tests**
   - `should_start_recording_on_mic_tap`: Verify recording starts
   - `should_stop_recording_on_stop_tap`: Test recording stops
   - `should_interrupt_on_button_press`: Verify interruption flow from UI

3. **Chat Display Tests**
   - `should_show_messages_with_proper_styling`: Test message appearance
   - `should_show_interruption_markers`: Verify interruption UI indicators
   - `should_scroll_to_new_messages`: Test auto-scrolling behavior
